In [1]:
import os
import h5py
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sbn
import datetime as dt
import matplotlib.pyplot as plt
from tqdm.notebook import  tqdm, trange

In [2]:
pts = gpd.read_file('../../CIAT/data/BEM_2019_Chiapas_Final_Clean.gpkg')
pts['SIM'] = ['CH{0:05d}'.format(i + 1) for i in range(len(pts))]
pts = pts.set_index('SIM')
pts['ref_date'] = pd.to_datetime(pts.fecha_de_siembra)
SIM = pts.index

In [3]:
w_np = np.zeros((len(SIM), 300, 6))
w_dm = np.zeros((len(SIM), 300, 6))
for i, key in enumerate(tqdm(SIM)):
    wfile = f'/home/rodrigo7/Apsim_test/MASAGRO/PCSE/{key}_npf.csv'
    save_file = f'/home/rodrigo7/Apsim_test/MASAGRO/PCSE/{key}_npff.csv'

    crop_start_date = pts.ref_date.dt.date.loc[key]
    sim_start_date = crop_start_date - dt.timedelta(days = 90)
    crop_end_date = crop_start_date + dt.timedelta(days = 210)
    
    w = pd.read_csv(wfile, skiprows=13)
    w.DAY = pd.to_datetime(w.DAY, format= '%Y%m%d').dt.date
    ini_idx = np.where(w.DAY == sim_start_date)[0][0]
    end_idx = np.where(w.DAY == crop_end_date)[0][0]
    s_idx = slice(ini_idx, end_idx)
    df_np = w.iloc[s_idx].copy()
    df_np.DAY = pd.to_datetime(df_np.DAY).dt.strftime('%Y%m%d')
    w_np[i] = df_np.values[:,1:-1]
    
    with open(save_file, 'w') as sf:
        with open(wfile) as f:
            for r in range(14):
                sf.writelines(f.readline())
    
    df_np.to_csv(save_file, na_rep = 'NaN', mode = 'a', float_format = '%.3f', header = False, index = False)

    #################################################################
    wfile = f'/home/rodrigo7/Apsim_test/MASAGRO/PCSE/{key}_dmf.csv'
    save_file = f'/home/rodrigo7/Apsim_test/MASAGRO/PCSE/{key}_dmff.csv'
    
    try:
        w = pd.read_csv(wfile, skiprows=13)
        w.DAY = pd.to_datetime(w.DAY, format= '%Y%m%d').dt.date
        ini_idx = np.where(w.DAY == sim_start_date)[0][0]
        end_idx = np.where(w.DAY == crop_end_date)[0][0]
        s_idx = slice(ini_idx, end_idx)
        df_dm = w.iloc[s_idx].copy()
        df_dm.DAY = pd.to_datetime(df_dm.DAY).dt.strftime('%Y%m%d')
    
    except:
        df_dm = df_np
        
    w_dm[i] = df_dm.values[:,1:-1]

    with open(save_file, 'w') as sf:
        with open(wfile) as f:
            for r in range(14):
                sf.writelines(f.readline())
    
    df_dm.to_csv(save_file, na_rep = 'NaN', mode = 'a', float_format = '%.3f', header = False, index = False)

np.save('../data/w_dm.npy', w_dm)
np.save('../data/w_np.npy', w_np)